<a href="https://colab.research.google.com/github/timeseriesAI/tsai/blob/master/tutorial_nbs/01_Intro_to_Time_Series_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

created by Ignacio Oguiza - email: timeseriesAI@gmail.com

## Purpose 😇

The purpose of this notebook is to show you how you can take any model or dataset in TSAI and run a hyperparameter optimization job to search for combination that yields the best result on the dataset.

## Import libraries 📚

In [1]:
# stable = True # True: latest version, False: stable version
# import sys
# ISCOLAB = 'google.colab' in sys.modules
# if ISCOLAB:
#     if stable: 
#         !pip install tsai -Uqq
#     else:
#         !pip install -Uqq git+https://github.com/timeseriesAI/tsai.git

In [2]:
import tsai
from tsai.all import *
print('tsai       :', tsai.__version__)
print('fastai     :', fastai.__version__)
print('fastcore   :', fastcore.__version__)
print('torch      :', torch.__version__)

tsai       : 0.2.19
fastai     : 2.5.0
fastcore   : 1.3.26
torch      : 1.9.0+cu102


## Baseline 📉

Before embarking on any hyperparameter optimization tasks, it is important to get a baseline performance so that we can note the improvements after the optimization is done.
In this notebook we use the InceptionTimePlus model and and train on the NATOPS dataset, both conveniently provided in TSAI in just few lines of codes. 
Feel free to use any other models and datasets.

In [23]:
dsid = 'NATOPS' 
X, y, splits = get_UCR_data(dsid, return_split=False)

# Old API
tfms  = [None, [Categorize()]]
dsets = TSDatasets(X, y, tfms=tfms, splits=splits, inplace=True)
dls   = TSDataLoaders.from_dsets(dsets.train, dsets.valid, bs=[64, 128], batch_tfms=[TSStandardize()], num_workers=0)
model = InceptionTimePlus(dls.vars, dls.c)
learn = Learner(dls, model, metrics=accuracy)
learn.fit_one_cycle(5, lr_max=1e-3)
learn.plot_metrics()

# New API
# learn = TSClassifier(X, y, splits=splits, bs=[64, 128], batch_tfms=[TSStandardize()], arch=InceptionTimePlus, metrics=accuracy)
# learn.fit_one_cycle(5, lr_max=1e-3)
# learn.plot_metrics()

## Install Optuna 🕹️

[Optuna]('https://optuna.readthedocs.io/en/stable/index.html') is an automatic hyperparameter optimization software framework, particularly designed for machine learning.

In [4]:
# !pip install optuna

## Define objective function 🎯

There are two components in the objective function that you need to define:
1. Search space - the hyperparameter values that you would like to search. In this example we are searching for the combination of nf and dropout rate.
2. Objective value - the value that will be used to indicate the performance of the model. In this example we use the validation loss as the objective value.

In [20]:
import optuna
from optuna.integration import FastAIPruningCallback # A callback to prune unpromising trials (ie early stopping) in Optuna. Optional.

def objective(trial:optuna.Trial):
    
    # Define search space here. Supported search space are 
    num_filters = trial.suggest_categorical('num_filters', [32, 64, 96, 128]) 
    dropout_rate = trial.suggest_float("dropout_rate", 0.0, 1.0)

    # Define the model with the search space value
    model = InceptionTimePlus(dls.vars, dls.c, nf=num_filters, fc_dropout=dropout_rate)
    learn = Learner(dls, model, metrics=accuracy, cbs=FastAIPruningCallback(trial))
    learn.fit_one_cycle(5, lr_max=1e-3)
       
    # Return the objective value
    # learn.recorder.values returns [train_loss, valid_loss, metrics] for every epoch.
    return learn.recorder.values[-1][1] # return the validation loss value of the last epoch 

## Start the study 🧑‍🎓

In Optuna, the hyperparameter search job is known as a Study. Each Study consists of many Trials. The number of Trials indicate how many times do you want Optuna to search through the search space.

After configuring the objective function above we would like to let Optuna to perform the search (study) for the combination of hyperparameters that yield the best objective value. 

Note: In the objective function we used the validation loss as our our objective value. Hence in the the study, we must tell Optuna minimize the objective value (This can be specified in the `direction='minimize'` as shown below).

Alternatively, if you chose to use the accuracy metric as the objective value defined in objective function, you will need to tell Optuna to maximize instead. (This can be specified in the `direction='maximi'` )

In [6]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=50)

[I 2021-08-10 12:39:57,114] A new study created in memory with name: no-name-c425bea2-bcbe-4549-88ba-d852895e24c6


[I 2021-08-10 12:39:57,482] Trial 0 finished with value: 1.6753536462783813 and parameters: {'num_filters': 32, 'dropout_rate': 0.7596492395710577}. Best is trial 0 with value: 1.6753536462783813.


[I 2021-08-10 12:39:57,920] Trial 1 finished with value: 1.6439212560653687 and parameters: {'num_filters': 32, 'dropout_rate': 0.5662686552152928}. Best is trial 1 with value: 1.6439212560653687.


[I 2021-08-10 12:40:00,358] Trial 2 finished with value: 1.1561634540557861 and parameters: {'num_filters': 96, 'dropout_rate': 0.27261062870632946}. Best is trial 2 with value: 1.1561634540557861.


[I 2021-08-10 12:40:01,071] Trial 3 finished with value: 1.3683940172195435 and parameters: {'num_filters': 96, 'dropout_rate': 0.8989755846980007}. Best is trial 2 with value: 1.1561634540557861.


...

[I 2021-08-10 12:40:22,117] Trial 45 pruned. Trial was pruned at epoch 0.


[I 2021-08-10 12:40:22,229] Trial 46 pruned. Trial was pruned at epoch 0.


[I 2021-08-10 12:40:23,204] Trial 47 finished with value: 0.8324723243713379 and parameters: {'num_filters': 128, 'dropout_rate': 0.11220162570946982}. Best is trial 33 with value: 0.794217586517334.


[I 2021-08-10 12:40:23,456] Trial 48 pruned. Trial was pruned at epoch 0.


[I 2021-08-10 12:40:23,882] Trial 49 pruned. Trial was pruned at epoch 1.


In [14]:
print("Study statistics: ")
print("  Number of finished trials: ", len(study.trials))

print("Best trial:")
trial = study.best_trial

print("  Value: ", trial.value)

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

Study statistics: 
  Number of finished trials:  50
Best trial:
  Value:  0.794217586517334
  Params: 
    num_filters: 128
    dropout_rate: 0.24388033387736385


## Training the optimized model

Now that we have obtained the optimized hyperparameters from the Optuna study, we can re-train the InceptionTimePlus model with the optimal hyperparameter values and note the improvement from the baseline model.

In [18]:
# Get the best nf and dropout rate from the best trial object

trial = study.best_trial
nf = trial.params['num_filters']
dropout_rate = trial.params['dropout_rate']

In [33]:
model = InceptionTimePlus(dls.vars, dls.c, nf=nf, fc_dropout=dropout_rate)
learn = Learner(dls, model, metrics=accuracy, cbs=FastAIPruningCallback(trial))

# New API
# learn = TSClassifier(X, y, splits=splits, bs=[64, 128], 
#                      batch_tfms=[TSStandardize()], metrics=accuracy,  
#                      arch=InceptionTimePlus, arch_config={'nf':nf, 'fc_dropout':dropout_rate})

learn.fit_one_cycle(5, lr_max=1e-3)
learn.plot_metrics()

For comparison our baseline model can only scored around 50% accuracy with 5 epochs training. However, using the hyperparameters from the Optuna study results in much higher accuracy. The numbers might vary due to the randomness in training. You can seed you runs or train the model a few times to verify the results. Happy learning! 